In [ ]:
%run main.py
%load_ext autoreload
%autoreload 2

!mkdir -p {DATA_DIR} {BERT_DIR} {MODEL_DIR}
s3 = S3()

In [ ]:
if not exists(NEWS):
    s3.download(S3_NEWS, NEWS)
    s3.download(S3_FICTION, FICTION)

In [ ]:
if not exists(BERT_VOCAB):
    s3.download(S3_BERT_VOCAB, BERT_VOCAB)
    s3.download(S3_BERT_EMB, BERT_EMB)
    s3.download(S3_BERT_ENCODER, BERT_ENCODER)

In [ ]:
words_vocab = BERTVocab.load(BERT_VOCAB)

In [ ]:
markups = {}
for path, name in [(NEWS, TEST), (FICTION, TRAIN)]:
    lines = load_gz_lines(path)
    items = parse_jl(lines)
    items = log_progress(items, desc=path)
    records = []
    for item in items:
        record = MorphMarkup.from_json(item)
        records.append(record)
    markups[name] = records

tags = set()
for name in [TEST, TRAIN]:
    for markup in markups[name]:
        for token in markup.tokens:
            tags.add(token.tag)
            
tags = [PAD] + sorted(tags)
tags_vocab = Vocab(tags)

In [ ]:
torch.manual_seed(SEED)
seed(SEED)

In [ ]:
config = RuBERTConfig()
emb = BERTEmbedding.from_config(config)
encoder = BERTEncoder.from_config(config)
morph = BERTMorphHead(config.emb_dim, len(tags_vocab))
model = BERTMorph(emb, encoder, morph)

for param in emb.parameters():
    param.requires_grad = False

model.emb.load(BERT_EMB)
model.encoder.load(BERT_ENCODER)
model = model.to(DEVICE)

criterion = masked_flatten_cross_entropy

In [ ]:
encode = BERTMorphTrainEncoder(
    words_vocab, tags_vocab,
    seq_len=128,
    batch_size=32,
    shuffle_size=10000
)

batches = {}
for name in [TEST, TRAIN]:
    batches[name] = [_.to(DEVICE) for _ in encode(markups[name])]

In [ ]:
board = MultiBoard([
    TensorBoard(BOARD_NAME, RUNS_DIR),
    LogBoard()
])
boards = {
    TRAIN: board.section(TRAIN_BOARD),
    TEST: board.section(TEST_BOARD),
}

In [ ]:
optimizer = optim.Adam([
    dict(params=encoder.parameters(), lr=BERT_LR),
    dict(params=morph.parameters(), lr=LR),
])
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, LR_GAMMA)

In [ ]:
meters = {
    TRAIN: MorphScoreMeter(),
    TEST: MorphScoreMeter()
}

for epoch in log_progress(range(EPOCHS)):
    model.train()
    for batch in log_progress(batches[TRAIN], leave=False):
        optimizer.zero_grad()
        batch = process_batch(model, criterion, batch)
        batch.loss.backward()
        optimizer.step()
    
        score = score_morph_batch(batch)
        meters[TRAIN].add(score)

    meters[TRAIN].write(boards[TRAIN])
    meters[TRAIN].reset()

    model.eval()
    with torch.no_grad():
        for batch in log_progress(batches[TEST], leave=False, desc=TEST):
            batch = process_batch(model, criterion, batch)
            score = score_morph_batch(batch)
            meters[TEST].add(score)
        meters[TEST].write(boards[TEST])
        meters[TEST].reset()
    
    scheduler.step()
    board.step()

In [ ]:
# [2020-03-31 14:26:22]    0 0.3800 01_train/01_loss
# [2020-03-31 14:26:22]    0 0.9177 01_train/02_acc
# [2020-03-31 14:26:22]    0 0.4991 02_test/01_loss
# [2020-03-31 14:26:22]    0 0.9356 02_test/02_acc
# [2020-03-31 14:27:44]    1 0.0919 01_train/01_loss
# [2020-03-31 14:27:44]    1 0.9740 01_train/02_acc
# [2020-03-31 14:27:44]    1 0.5095 02_test/01_loss
# [2020-03-31 14:27:44]    1 0.9388 02_test/02_acc
# [2020-03-31 14:29:06]    2 0.0655 01_train/01_loss
# [2020-03-31 14:29:06]    2 0.9808 01_train/02_acc
# [2020-03-31 14:29:06]    2 0.5176 02_test/01_loss
# [2020-03-31 14:29:06]    2 0.9429 02_test/02_acc
# [2020-03-31 14:30:28]    3 0.0463 01_train/01_loss
# [2020-03-31 14:30:28]    3 0.9861 01_train/02_acc
# [2020-03-31 14:30:28]    3 0.5389 02_test/01_loss
# [2020-03-31 14:30:28]    3 0.9423 02_test/02_acc
# [2020-03-31 14:31:50]    4 0.0344 01_train/01_loss
# [2020-03-31 14:31:50]    4 0.9895 01_train/02_acc

# [2020-03-31 14:31:50]    4 0.5649 02_test/01_loss
# [2020-03-31 14:31:50]    4 0.9416 02_test/02_acc

In [ ]:
# model.encoder.dump(MODEL_ENCODER)
# model.morph.dump(MODEL_MORPH)
# tags_vocab.dump(TAGS_VOCAB)
        
# s3.upload(MODEL_ENCODER, S3_MODEL_ENCODER)
# s3.upload(MODEL_MORPH, S3_MODEL_MORPH)
# s3.upload(TAGS_VOCAB, S3_TAGS_VOCAB)